In [56]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("data/transaction_fraud_data.parquet")
data.head()

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_card_present,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,False,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,False,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [5]:
data['is_fraud'].mean()

np.float64(0.19972818498066347)

In [10]:
data.groupby(by='country')['is_fraud'].sum().sort_values()

country
Germany       37205
Canada        37278
USA           37312
UK            37345
Singapore     37414
France        37426
Japan         37592
Australia     37652
Nigeria      298600
Brazil       298629
Mexico       298841
Russia       299425
Name: is_fraud, dtype: int64

# ??????


In [13]:
data['last_hour_activity'].apply(lambda x: x['num_transactions']).mean()

np.float64(409.14292416411735)

In [15]:
data[data['is_high_risk_vendor'] == True]['is_fraud'].mean()

np.float64(0.19998632451602943)

In [17]:
data.groupby(by='city')['amount'].mean().sort_values()

city
Dallas            562.399533
Philadelphia      563.464431
Los Angeles       563.900769
Houston           564.961388
Chicago           565.277374
San Diego         565.593140
Phoenix           565.730142
San Jose          567.263760
San Antonio       567.921894
New York          568.872467
Unknown City    51315.780071
Name: amount, dtype: float64

In [18]:
data['vendor_category'].unique()

array(['Restaurant', 'Entertainment', 'Grocery', 'Gas', 'Healthcare',
       'Education', 'Travel', 'Retail'], dtype=object)

In [7]:
data[data['vendor_type'] == 'fast_food'].groupby(by='city')['amount'].mean().sort_values()

city
Phoenix           242.334715
Philadelphia      243.440463
Houston           246.913137
San Jose          247.892502
Dallas            248.685716
San Diego         254.343884
Los Angeles       255.950324
San Antonio       258.169691
New York          263.296441
Chicago           264.450281
Unknown City    15214.407916
Name: amount, dtype: float64

In [3]:
data_cur = pd.read_parquet('data\historical_currency_exchange.parquet')

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
C:\Users\mrcho\AppData\Local\Temp\ipykernel_15016\2143820909.py:1: SyntaxWarning: invalid escape sequence '\h'
  data_cur = pd.read_parquet('data\historical_currency_exchange.parquet')


In [4]:
data_cur.head()

,date,AUD,BRL,CAD,EUR,GBP,JPY,MXN,NGN,RUB,SGD,USD
0,2024-09-30,1.443654,5.434649,1.351196,0.895591,0.747153,142.573268,19.694724,1668.736400,94.133735,1.280156,1
1,2024-10-01,1.442917,5.444170,1.352168,0.897557,0.746956,143.831429,19.667561,1670.694524,92.898519,1.284352,1
2,2024-10-02,1.449505,5.425444,1.348063,0.903056,0.752241,143.806861,19.606748,1669.653006,94.583198,1.286983,1
3,2024-10-03,1.456279,5.442044,1.351451,0.906018,0.754584,146.916773,19.457701,1670.097873,95.655442,1.294391,1
4,2024-10-04,1.460930,5.477788,1.355260,0.906452,0.761891,146.592323,19.363467,1649.763738,94.755337,1.296800,1


In [23]:
data['date'] = data['timestamp'].dt.date

In [39]:
data_joined = data.merge(data_cur, on='date', how='left')

In [41]:
data_joined.columns

Index(['transaction_id', 'customer_id', 'card_number', 'timestamp',
       'vendor_category', 'vendor_type', 'vendor', 'amount', 'currency',
       'country', 'city', 'city_size', 'card_type', 'is_card_present',
       'device', 'channel', 'device_fingerprint', 'ip_address',
       'is_outside_home_country', 'is_high_risk_vendor', 'is_weekend',
       'last_hour_activity', 'is_fraud', 'date', 'AUD', 'BRL', 'CAD', 'EUR',
       'GBP', 'JPY', 'MXN', 'NGN', 'RUB', 'SGD', 'USD'],
      dtype='object')

In [ ]:
data_joined['amount_usd'] = data_joined.apply(
    lambda row: row['amount'] / row[row['currency']] if row['currency'] != 'USD' else row['amount'],
    axis=1
)

In [49]:
data_joined[data_joined['is_fraud'] == False]['amount_usd'].std(ddof=1)

np.float64(417.0077334756039)

In [50]:
data_joined[data_joined['is_fraud'] == True]['amount_usd'].mean()

np.float64(874.6067503080814)

In [53]:
data_joined[data_joined['is_fraud'] == True]['amount_usd'].std(ddof=1)

np.float64(1349.882692206744)

In [59]:
prep_data = data.copy()
prep_data['last_hour_activity'] = data['last_hour_activity'].apply(lambda x: x['unique_merchants'])

In [61]:
customers = prep_data.groupby(by='customer_id')['last_hour_activity'].median()

In [65]:
sum(customers > customers.quantile(0.95))

229

In [68]:
data['hour'] = data['timestamp'].dt.hour
transactions_per_hour = data.groupby(['customer_id', 'hour'])['transaction_id'].count().reset_index()


avg_transactions_per_hour = transactions_per_hour.groupby('customer_id')['transaction_id'].mean().mean()

avg_transactions_per_hour

np.float64(64.04263366878895)

In [67]:
transactions_per_hour

,customer_id,hour,transaction_id
0,CUST_10000,0,28
1,CUST_10000,1,55
2,CUST_10000,2,53
3,CUST_10000,3,49
4,CUST_10000,4,57
...,...,...,...
116851,CUST_99971,19,103
116852,CUST_99971,20,74
116853,CUST_99971,21,42
116854,CUST_99971,22,106
